In [1]:
import os
import pandas as pd

In [3]:
def getComparisonDF(etf_name):
    iceComparison_df = pd.read_excel('Output/ICE/{} Rebalancing Numbers.xlsx'.format(etf_name), sheet_name='{} - ICE Index'.format(etf_name))
    iceComparison_df = iceComparison_df[['ISIN number', 'Difference Today', 'Ticker']]
    iceComparison_df.rename(columns={'Difference Today':'ICE Difference'}, inplace=True)
    
    etfComparison_df = pd.read_excel('Output/ETF/{} ETF vs ICE.xlsx'.format(etf_name), sheet_name=etf_name)
    if('ISIN number' in etfComparison_df.columns):
        etfComparison_df.rename(columns={'ISIN number':'ISIN'}, inplace=True)
    etfComparison_df = etfComparison_df[['ISIN', 'Difference']]
    etfComparison_df.rename(columns={'Difference':'ETF Difference'}, inplace=True)
    
    merged_df = iceComparison_df.merge(etfComparison_df, left_on='ISIN number', right_on='ISIN', how='outer')
    merged_df['ICE vs ETF'] = merged_df.apply(lambda x: x['ICE Difference']-x['ETF Difference'], axis=1)
    merged_df.sort_values(by='ICE vs ETF', key=abs, ascending=False, inplace=True)
    
    return merged_df

In [4]:
def getFileWriter(outputFile):
    '''Return Append Mode if file is present else Write Mode '''
    if(os.path.isfile(outputFile)):
        return pd.ExcelWriter(outputFile, mode="a", if_sheet_exists="replace", engine='openpyxl')
    return pd.ExcelWriter(outputFile, mode="w", engine='openpyxl')

In [5]:
def writeToSingleFile(merged_df, etf_name):
    outputFile = 'Output/ICE vs ETF.xlsx'
    writer = getFileWriter(outputFile) 
    
    merged_df.to_excel(writer, sheet_name='{}'.format(etf_name))
    writer.close()

In [6]:
def main():
    
    etf_list = ['PGX', 'PFF', 'PSK', 'PGF', 'PFFD', 'PFFV']
    
    for etf_name in etf_list:
        print(etf_name)
        merged_df = getComparisonDF(etf_name)
        writeToSingleFile(merged_df, etf_name)

In [7]:
main()

PGX
PFF
PSK
PGF
PFFD
PFFV
